## Latar Belakang

Portugal merupakan salah satu negara yang terletak di ujung barat daya Eropa. Banyak wisatwan yang berkunjung ke portugal untuk berlibur, tapi ada juga yang berkunjung untuk urusan bisnis atau yang lain nya. Selama berkunjung pelanggan akan menginap di hotel. Karena banyak nya pengunjung yang memesan hotel, maka banyak pengunjung yang memesan hotel dari jauh hari. Namun setelah pemesanan banyak pelanggan yang tiba-tiba mengcancel pesanannya.

Maka dalam kesempatan ini saya akan mencoba membuat sistem yang dapat memprediksi pelanggan mana yang akan cancel menggunakan machine learning

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from imblearn.over_sampling import RandomOverSampler,SMOTE
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score,accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier,StackingClassifier,RandomForestClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,RobustScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [3]:
Data_Raw = pd.read_csv('hotel_bookings.csv')
Data_1 = Data_Raw

In [4]:
Data_1.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### Data Understanding

Yang menjadi target (y) = is_canceled  dimana 1 adalah yang cancel pesanan dan 0 sebaliknya. untuk kasus ini fokus nya adalah yang cancel sehingga dalam pengunaan metrix lebih mengutamakan recall

Yang menjadi Feature (x) = 'hotel','lead_time','country','market_segment',  
'customer_type','days_in_waiting_list','booking_changes','previous_cancellations',  
'previous_bookings_not_canceled',  'is_repeated_guest'

### Data Processing

In [5]:
Data_1.shape

(119390, 32)

Jumlah Kolom = 32  
Jumlah Baris = 119390

### Data Duplikat

In [6]:
Data_1[Data_1.duplicated()].shape

(31994, 32)

terdapat 31994 data duplikat yang harus di drop karena jikat tidak drop akan membuat pemodelan menjadi bias

In [7]:
Data_1 = Data_1[~Data_1.duplicated()]

### Feature Selection

In [8]:
feature = ['hotel','lead_time','country','market_segment','customer_type','days_in_waiting_list','booking_changes','previous_cancellations','previous_bookings_not_canceled','is_repeated_guest','is_canceled']
Data_1 = Data_1[feature]

### Data NAN

In [22]:
Data_1.isna().sum()

hotel                               0
lead_time                           0
country                           452
market_segment                      0
customer_type                       0
days_in_waiting_list                0
booking_changes                     0
previous_cancellations              0
previous_bookings_not_canceled      0
is_repeated_guest                   0
is_canceled                         0
dtype: int64

Di kolom country terdapat 452 data nan

In [23]:
Data_1['country'].value_counts()

country
PRT    27453
GBR    10433
FRA     8837
ESP     7252
DEU     5387
       ...  
MMR        1
BFA        1
CYM        1
MLI        1
KHM        1
Name: count, Length: 177, dtype: int64

Karena mayoritas pelanggan berasal dari Portugal(PRT) maka data nan akan diisi dengan PRT

In [9]:
Data_1['country'].fillna('PRT',inplace=True)

In [9]:
Data_1.isna().sum()

hotel                             0
lead_time                         0
country                           0
market_segment                    0
customer_type                     0
days_in_waiting_list              0
booking_changes                   0
previous_cancellations            0
previous_bookings_not_canceled    0
is_repeated_guest                 0
is_canceled                       0
dtype: int64

### Perbandingan Data

In [26]:
display(Data_1['is_canceled'].value_counts())
Data_1['is_canceled'].value_counts()/len(Data_1)

is_canceled
0    63371
1    24025
Name: count, dtype: int64

is_canceled
0    0.725102
1    0.274898
Name: count, dtype: float64

Terdapat ketidak seimbangan data antara yang 0 dan 1 sebesar 72/28 yang nanti untuk penangulangan nya bisa dilakukan resambling

### Train Test Split

In [10]:
X = Data_1.drop("is_canceled",axis=1)
y = Data_1['is_canceled']

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

### PreProcess

Untuk PreProcess data tools yang digunakan adalah:  
1. One Hot Encode, Kolom yang digunakan untuk diencode adalah 'hotel'
2. Binary Encode, Kolom yang digunakan untuk diencode adalah 'country','market_segment','customer_type'
3. Robust Scaler, Kolom yang digunakan untuk diskaling adalah 'lead_time','days_in_waiting_list','booking_changes','previous_cancellations','previous_bookings_not_canceled'
4. SMOTE, digunakan untuk oversampling data karena adanya imbalance data

In [12]:
prepros = ColumnTransformer([
    ('One Hot Encode',OneHotEncoder(drop='first'),['hotel']),
    ('Binary Encode',ce.BinaryEncoder(),['country','market_segment','customer_type']),
    ('Robust Scaler',RobustScaler(),['lead_time','days_in_waiting_list','booking_changes','previous_cancellations','previous_bookings_not_canceled'])
],remainder='passthrough')

In [13]:
oversample = SMOTE(random_state=42)

### Modelling

Algoritma yang digunakan:
1. logistic regressi
2. KNN
3. Decision Tree
4. Random Forest
5. XGBoost
6. Voting Classifier
7. Stacking Classifier

In [14]:
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier(n_neighbors=25)
tree = DecisionTreeClassifier(criterion='gini',max_depth=3)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
XGB = XGBClassifier()

In [15]:
# Voting Classifier

vc_hard = VotingClassifier([
    ('Logreg',logreg),
    ('KNN',knn),
    ('DTree',tree),
    ('Random Forrest',rf_classifier),
    ('XGB',XGB)
],voting='hard')

vc_soft = VotingClassifier([
    ('Logreg',logreg),
    ('KNN',knn),
    ('DTree',tree),
    ('Random Forrest',rf_classifier),
    ('XGB',XGB)
],voting='soft')

In [16]:
# Stacking Classifier
stacking = StackingClassifier([
    ('Logreg',logreg),
    ('KNN',knn),
    ('DTree',tree),
    ('Random Forrest',rf_classifier),
    ('XGB',XGB)
])

### Model Benchmarking

Di sini akan dilakukan benchmarking untuk membandingkan algorithm mana yang paling terbaik.  
Algoritma terbaik itu lah yang nanti akan dikembangkan lebih lanjut untuk bisa mendapat hasil yang optimal

In [34]:
model = [logreg,knn,tree,rf_classifier,XGB,vc_hard,vc_soft,stacking]

skfold = StratifiedKFold(n_splits=5)

score_acc = [] # list untuk mengimpan score accuracy dari setiap algoritma
score_rec = [] # list untuk mengimpan score recall dari setiap algoritma
score_f1 = [] # list untuk mengimpan score F1 dari setiap algoritma
score_pre = [] # list untuk mengimpan score precision dari setiap algoritma


for i in model:
    pipeline = Pipeline([ # Di sini akan dibuat pipeline 
        ('Preprocessor', prepros), # Langkah pertama adalah preproses data
        ('Sampling',oversample), # Langkah kedua adalah oversampling data
        ('Model', i) # Langkah ketiga adalah pemodelan
        ])
    model_acc = cross_val_score(pipeline,X_train,y_train,cv = skfold, scoring='accuracy') # Disini akan dilakukan cross validasi yang akan mendapatkan nilai accuracy dari setiap algoritma
    model_rec = cross_val_score(pipeline,X_train,y_train,cv = skfold, scoring='recall') # Disini akan dilakukan cross validasi yang akan mendapatkan nilai recall dari setiap algoritma
    model_f1 = cross_val_score(pipeline,X_train,y_train,cv = skfold, scoring='f1') # Disini akan dilakukan cross validasi yang akan mendapatkan nilai F1 dari setiap algoritma
    model_pre = cross_val_score(pipeline,X_train,y_train,cv = skfold, scoring='precision') # Disini akan dilakukan cross validasi yang akan mendapatkan nilai presicion dari setiap algoritma
    score_acc.append(model_acc)
    score_rec.append(model_rec)
    score_f1.append(model_f1)
    score_pre.append(model_pre)


In [35]:
dfKfold = pd.DataFrame({
    'Model': ['Logistic Regression','KNN','Decision Tree','Random Forrest','XGB','VC_hard','VC_soft','Stacking'],
    'Avg Accuracy':[i.mean() for i in score_acc],
    'Avg Recall':[i.mean() for i in score_rec],
    'Avg Precision':[i.mean() for i in score_f1],
    'Avg F1':[i.mean() for i in score_pre]
}
)

In [51]:
dfKfold1 = dfKfold
dfKfold1.sort_values('Avg Recall',ascending=False) # untuk train test algoritma dengan recall tertinggi adalah Decision Tree dan XGB

,Model,Avg Accuracy,Avg Recall,Avg Precision,Avg F1
2,Decision Tree,0.607243,0.863163,0.547178,0.400585
4,XGB,0.708379,0.825338,0.608769,0.482267
5,VC_hard,0.716202,0.801717,0.608323,0.490151
6,VC_soft,0.728446,0.764984,0.607659,0.504048
1,KNN,0.714071,0.743600,0.588439,0.486877
7,Stacking,0.738901,0.702133,0.596511,0.518576
0,Logistic Regression,0.684593,0.693652,0.547345,0.452039
3,Random Forrest,0.732994,0.614204,0.558427,0.511959


In [37]:
# Disini akan dilakukan benchmarking kembali untuk data test yang akan berfokus pada recall
recall_1 = []
for i in model:
    pipa = Pipeline([
        ('Preprocessor', prepros),
        ('Model', i)
        ])
    pipa.fit(X_train,y_train)
    y_predict = pipa.predict(X_test)
    recall_1.append(recall_score(y_test,y_predict))

recallDF = pd.DataFrame({
    'Model':['Logistic Regression','KNN','Decision Tree','Random Forrest','XGB','VC_hard','VC_soft','Stacking'],
    'Recall':recall_1,
}
)  

In [48]:
recallDF.sort_values('Recall',ascending=False) 
# untuk data test, algoritma Random Forrest adalah yang terbaik sehingga untuk proses tuning akan fokus pada kedua algoritma

,Model,Recall
3,Random Forrest,0.489490
7,Stacking,0.462643
4,XGB,0.445369
1,KNN,0.435172
6,VC_soft,0.397503
5,VC_hard,0.371072
0,Logistic Regression,0.280541
2,Decision Tree,0.004579


### Uji Pengembangan Random Forrest

1. Uji criterion

In [17]:
entro = ['gini','entropy', 'log_loss']
hasil_rf = []
for i in entro:
        random_forest = Pipeline([
                ('Preprocessor', prepros),
                ('Model', RandomForestClassifier(criterion=i))
                ])
        random_forest.fit(X_train,y_train)
        y_predict = random_forest.predict(X_test)
        print(i)
        print(classification_report(y_test,y_predict))

gini
              precision    recall  f1-score   support

           0       0.82      0.86      0.84     12675
           1       0.57      0.49      0.53      4805

    accuracy                           0.76     17480
   macro avg       0.69      0.68      0.68     17480
weighted avg       0.75      0.76      0.75     17480

entropy
              precision    recall  f1-score   support

           0       0.82      0.86      0.84     12675
           1       0.57      0.49      0.53      4805

    accuracy                           0.76     17480
   macro avg       0.69      0.67      0.68     17480
weighted avg       0.75      0.76      0.75     17480

log_loss
              precision    recall  f1-score   support

           0       0.82      0.86      0.84     12675
           1       0.57      0.49      0.53      4805

    accuracy                           0.76     17480
   macro avg       0.69      0.67      0.68     17480
weighted avg       0.75      0.76      0.75     1748

### Kesimpulan Criterion
Ditemukan dari percobaan untuk setiap ketiga criterion, nilai recall tidak mengalami perubahan  

------
## Kesimpulan Akhir

Ditemukan bahwa untuk memprediksi pelanggan yang akan mengcancel pemesanan hotel lebih sulit dari pada yang tidak mengcancel yang disebabkan adanya imbalance data antara yang tidak cancel dan cancel sebesar 72/28. Setelah melakukan bencmarking dan tunning pun didapatkan kemampuan model untuk memperediksi yang cancel hanya sebesar 49% atau recall sebesar = 0.49 dimana recall berfokus pada ketepatan model memperediksi nilai 1 atau cancel

-----
## Uji Coba

Kali ini akan mencoba dengan dataset baru untuk memperediksi apakah pelanggan akan cancel atau tidak

In [22]:
feature.pop()

'is_canceled'

In [24]:
Data_baru = pd.read_csv('Data Baru - Train Dataset.csv')
Data_baru = Data_baru[feature]

In [25]:
Data_baru

,hotel,lead_time,country,market_segment,customer_type,days_in_waiting_list,booking_changes,previous_cancellations,previous_bookings_not_canceled,is_repeated_guest
0,Resort Hotel,342,PRT,Direct,Transient,0,3,0,0,0
1,Resort Hotel,737,PRT,Direct,Transient,0,4,0,0,0
2,Resort Hotel,7,GBR,Direct,Transient,0,0,0,0,0
3,Resort Hotel,13,GBR,Corporate,Transient,0,0,0,0,0
4,Resort Hotel,14,GBR,Online TA,Transient,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1936,Resort Hotel,114,PRT,Groups,Contract,0,0,0,0,0
1937,Resort Hotel,114,PRT,Groups,Transient,0,0,0,0,0
1938,Resort Hotel,0,PRT,Groups,Transient,0,0,0,0,0
1939,Resort Hotel,7,PRT,Corporate,Transient,0,0,0,0,0


In [26]:
y_predict = random_forest.predict(Data_baru)

In [28]:
Data_baru['is_canceled'] = y_predict
Data_baru

,hotel,lead_time,country,market_segment,customer_type,days_in_waiting_list,booking_changes,previous_cancellations,previous_bookings_not_canceled,is_repeated_guest,is_canceled
0,Resort Hotel,342,PRT,Direct,Transient,0,3,0,0,0,0
1,Resort Hotel,737,PRT,Direct,Transient,0,4,0,0,0,0
2,Resort Hotel,7,GBR,Direct,Transient,0,0,0,0,0,0
3,Resort Hotel,13,GBR,Corporate,Transient,0,0,0,0,0,0
4,Resort Hotel,14,GBR,Online TA,Transient,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1936,Resort Hotel,114,PRT,Groups,Contract,0,0,0,0,0,0
1937,Resort Hotel,114,PRT,Groups,Transient,0,0,0,0,0,1
1938,Resort Hotel,0,PRT,Groups,Transient,0,0,0,0,0,1
1939,Resort Hotel,7,PRT,Corporate,Transient,0,0,0,0,0,0


In [29]:
Data_baru['is_canceled'].value_counts()

is_canceled
0    1087
1     854
Name: count, dtype: int64

Untuk prediksi data baru ditemukan yang tidak cancel sebanyak 1087 dan yang cancel 854